**Scrape Spotrac Data**

In [1]:
#imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#source cited at bottom, use to create get_player_link and get_player_name functions below
def findnth(string, substring, n):
    parts = string.split(substring, n + 1)
    if len(parts) <= n + 1:
        return -1
    return len(string) - len(parts[-1]) - len(substring)

In [3]:
#function to get player link from row with player info in table
def get_player_link(var):
    player_link = var[findnth(var, 'h', 1):findnth(var, '/', 5)+1]
    return player_link

In [4]:
#function to get player name from row with player info in table
def get_player_name(var):
    player_name = var[findnth(var, '>', 0)+1:findnth(var, '<', 1)]
    return player_name

In [5]:
#blank df
df = pd.DataFrame()

In [6]:
#list of links to team cap pages
teams = ['https://www.spotrac.com/nfl/arizona-cardinals/cap/2021/', 'https://www.spotrac.com/nfl/atlanta-falcons/cap/2021/', 'https://www.spotrac.com/nfl/baltimore-ravens/cap/2021/', 'https://www.spotrac.com/nfl/buffalo-bills/cap/2021/', 'https://www.spotrac.com/nfl/carolina-panthers/cap/2021/', 'https://www.spotrac.com/nfl/chicago-bears/cap/2021/', 'https://www.spotrac.com/nfl/cincinnati-bengals/cap/2021/', 'https://www.spotrac.com/nfl/cleveland-browns/cap/2021/', 'https://www.spotrac.com/nfl/dallas-cowboys/cap/2021/', 'https://www.spotrac.com/nfl/denver-broncos/cap/2021/', 'https://www.spotrac.com/nfl/detroit-lions/cap/2021/', 'https://www.spotrac.com/nfl/green-bay-packers/cap/2021/', 'https://www.spotrac.com/nfl/houston-texans/cap/2021/', 'https://www.spotrac.com/nfl/indianapolis-colts/cap/2021/', 'https://www.spotrac.com/nfl/jacksonville-jaguars/cap/2021/', 'https://www.spotrac.com/nfl/kansas-city-chiefs/cap/2021/', 'https://www.spotrac.com/nfl/las-vegas-raiders/cap/2021/', 'https://www.spotrac.com/nfl/los-angeles-chargers/cap/2021/', 'https://www.spotrac.com/nfl/los-angeles-rams/cap/2021/', 'https://www.spotrac.com/nfl/miami-dolphins/cap/2021/', 'https://www.spotrac.com/nfl/minnesota-vikings/cap/2021/', 'https://www.spotrac.com/nfl/new-england-patriots/cap/2021/', 'https://www.spotrac.com/nfl/new-orleans-saints/cap/2021/', 'https://www.spotrac.com/nfl/new-york-giants/cap/2021/', 'https://www.spotrac.com/nfl/new-york-jets/cap/2021/', 'https://www.spotrac.com/nfl/philadelphia-eagles/cap/2021/', 'https://www.spotrac.com/nfl/pittsburgh-steelers/cap/2021/', 'https://www.spotrac.com/nfl/san-francisco-49ers/cap/2021/', 'https://www.spotrac.com/nfl/seattle-seahawks/cap/2021/', 'https://www.spotrac.com/nfl/tampa-bay-buccaneers/cap/2021/', 'https://www.spotrac.com/nfl/tennessee-titans/cap/2021/', 'https://www.spotrac.com/nfl/washington-football-team/cap/2021/']

In [7]:
#loop to find team cap pages with desired info, put all players into list players to iterate through
for team in teams:
    #scrape players tables from team pages
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    options = Options()
    options.add_argument('--disable-blink-features=AutomationControlled')
    driver.get(team) #using links from teams list
    soup = BeautifulSoup(driver.page_source)
    driver.quit()
    
    table = soup.find_all('table')[0] #This is the index of the table with all players
    players = table.find_all("a") #players stored in hyperlinked elements
    players = [str(player) for player in players] #convert list elements into strings
    players = [player for player in players if 'https' in player] #only keep strings that link to a player page
    
    #loop to find desired player salary info, put all info into df 
    for player in players:
        #scrape salary info from player pages
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        options = Options()
        options.add_argument('--disable-blink-features=AutomationControlled')
        driver.get(get_player_link(player))#using function to get player links and visiting those links
        

        popup = driver.find_element('xpath', '//*[@id="PopupSignupForm_0"]/div[2]/div[1]')#disabling pop-up
        popup.click()

        button = driver.find_element("xpath", '//*[@id="cash-earnings"]')#clicking on link from player link to go to desired page
        button.click()

        get_url = driver.current_url#return url from career earnings page

        response = urlopen(get_url)
        html = response.read()

        soup = BeautifulSoup(html)
        driver.quit()

        earnings_table = soup.find('table', {'class': 'earningstable rtable'}) #table with desired salary info
        tab_data = [[celldata.text for celldata in rowdata.find_all(["th","td"])]
                            for rowdata in earnings_table.find_all("tr")]
        earnings_table_rows = earnings_table.find_all('tr')#rows of table
        team_list = []#create list of team names
        #function converts team images that represent team on webpage to strings that can be used to represent team in df
        for row in earnings_table_rows[1:]:
            img = row.find('img')
            try:
                team_list.append(img['src'].split('.png')[0].rsplit('/', 1)[1].upper())
            except:
                team_list.append('')

        player_df = pd.DataFrame(tab_data[1:], columns = tab_data[0]) #first row is labels, creates df for each player with desired salary data
        player_df.insert(loc=0, column='Player', value=get_player_name(player)) #creates column to identify player by name, to link with pfr data later
        
        player_df['Team'] = team_list #creates column to identify player by team
        #gets rid of characters that would obscure numbers
        player_df = player_df.apply(lambda x: x.str.replace(',', ''))
        player_df = player_df.apply(lambda x: x.str.replace('$', ''))
        
        
        player_df['Year'] = pd.to_numeric(player_df['Year'],errors='coerce') #converts year to number, raise errors for strings that don;t represent a year
        player_df = player_df.dropna() #drop columns that don't represent a year, like totals
        player_df['Year'] = player_df['Year'].astype('int64') #convert year to integer
        
        #convert strings to numbers
        player_df[['Year', 'Salary', 'Signing Bonus', 'Roster Bonus', 'Workout Bonus', 'Restructure Bonus', 'Option Bonus', 'Incentive', 'Earnings']] = player_df[['Year', 'Salary', 'Signing Bonus', 'Roster Bonus', 'Workout Bonus', 'Restructure Bonus', 'Option Bonus', 'Incentive', 'Earnings']].apply(pd.to_numeric, errors='ignore')
        #add individual player contract info to df
        df = df.append(player_df)

In [8]:
#reset index of resulting df
df = df.reset_index()
df = df.drop(['index'], axis=1)

#some team pngs had numbers to differentiate logos, get rid of those so all Team column is just team abbreviations
df['Team'] = df['Team'].apply(lambda x: x.replace('_', ''))
df['Team'] = df['Team'].apply(lambda x: x.replace('0', ''))
df['Team'] = df['Team'].apply(lambda x: x.replace('1', ''))
df['Team'] = df['Team'].apply(lambda x: x.replace('2', ''))
df['Team'] = df['Team'].apply(lambda x: x.replace('3', ''))

In [19]:
#export resulting df to csv
df.to_csv(r'C:\Users\FIDataSci.DESKTOP-5QIEL80\spotrac-dataframe-csv', sep='\t', encoding='utf-8')

find nth function : https://www.tutorialspoint.com/How-to-find-the-nth-occurrence-of-substring-in-a-string-in-Python#:~:text=You%20can%20find%20the%20nth,occurs%20more%20than%20n%20times.